# Import packages

In [1]:
import numpy as np
from numpy import array
from pickle import dump
import os
import zipfile

# pandas is a data analysis tool, contains object DataFrame
import pandas as pd
#import torch
#
import random

# regex
import re

# datetime objects
import datetime

# plotting packages
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

# used for creating the dictionary
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# used for flattening lists 
from itertools import chain
from itertools import compress

# NLTK and WordNet lemmatizer
import nltk
from nltk.stem import WordNetLemmatizer



import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.layers.core import Dense, SpatialDropout1D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

from sklearn.model_selection import train_test_split
import numpy
import itertools
import json
from sklearn.model_selection import train_test_split
import sys, os, re, csv, codecs, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

from keras.layers import Input, Dense, LSTM, Flatten, concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras_self_attention import SeqSelfAttention
import numpy as np 
import numpy as np
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt

# load and evaluate a saved model
from numpy import loadtxt
from keras.models import load_model
from itertools import repeat
from keras.callbacks import Callback
 

# Training Data

In [2]:
train_data= pd.read_pickle("C:\\Users\\A373503\\Desktop\\Complete_4_models\\HDFS\\Data\\HDFS_train_mix_data.pkl")
word2id=pd.read_pickle('C:\\Users\\A373503\\Desktop\\Complete_4_models\\HDFS\\Data\\word2id_train.pkl')


In [3]:
train_data

,Blockid,EventSequence,labels,Length
569963,blk_-8263499994297877702,"[2, 2, 2, 12, 3, 4, 3, 4, 3, 4, 5, 5, 5, 6, 6,...",0,19
486412,blk_6623683650071326961,"[2, 11, 2, 2, 3, 4, 3, 4, 3, 4, 5, 5, 14]",0,13
171779,blk_2082025639925650040,"[2, 2, 2, 12, 3, 4, 3, 4, 3, 4, 5, 5, 5, 9, 10...",0,25
224584,blk_5448888175997650632,"[2, 2, 11, 2, 3, 4, 3, 4, 3, 4, 5, 5, 5, 6, 6,...",0,19
536913,blk_4492763112427066795,"[2, 2, 11, 2, 3, 4, 3, 4, 3, 4, 5, 5, 14]",0,13
...,...,...,...,...
460148,blk_1763114267597682207,"[2, 11, 2, 2, 5, 3, 4, 3, 4, 3, 4, 5, 5, 6, 6,...",0,19
111571,blk_-8078410045005505674,"[11, 2, 2, 2, 5, 5, 5, 3, 4, 3, 4, 3, 4, 13, 1...",0,25
462281,blk_-6694252843752535345,"[2, 2, 11, 24]",1,4
75704,blk_-8368340709610034828,"[12, 2, 2, 2, 5, 5, 5, 3, 4, 3, 4, 3, 4, 9, 9,...",0,31


In [4]:
print("the size of the data:",len(train_data))
print("the size of the vocab:",len(word2id))
train_data.groupby(['labels']).count()

the size of the data: 460048
the size of the vocab: 97


,Blockid,EventSequence,Length
labels,,,
0,446541,446541,446541
1,13507,13507,13507


In [5]:
word2id['unknown']=len(word2id)+1

In [6]:
print("the size of the updated vocab:",len(word2id))

the size of the updated vocab: 98


In [7]:
train_data.head()

,Blockid,EventSequence,labels,Length
569963,blk_-8263499994297877702,"[2, 2, 2, 12, 3, 4, 3, 4, 3, 4, 5, 5, 5, 6, 6,...",0,19
486412,blk_6623683650071326961,"[2, 11, 2, 2, 3, 4, 3, 4, 3, 4, 5, 5, 14]",0,13
171779,blk_2082025639925650040,"[2, 2, 2, 12, 3, 4, 3, 4, 3, 4, 5, 5, 5, 9, 10...",0,25
224584,blk_5448888175997650632,"[2, 2, 11, 2, 3, 4, 3, 4, 3, 4, 5, 5, 5, 6, 6,...",0,19
536913,blk_4492763112427066795,"[2, 2, 11, 2, 3, 4, 3, 4, 3, 4, 5, 5, 14]",0,13


In [8]:
train_data[train_data['Length']<10]

,Blockid,EventSequence,labels,Length
549142,blk_9030539814686132531,"[2, 2, 12, 24]",1,4
292808,blk_7717720559218514250,"[2, 32]",1,2
87956,blk_-4397275276774729449,"[12, 27]",1,2
62615,blk_6429121745702481909,"[15, 27]",1,2
52885,blk_5567058207076986877,"[15, 2, 2, 24]",1,4
...,...,...,...,...
7209,blk_7165939739964393997,"[11, 2, 2, 24]",1,4
424987,blk_-6475794039385849761,"[2, 33]",1,2
545982,blk_-6810082241294284349,"[2, 33]",1,2
428480,blk_-2944709241310761499,"[2, 2, 12, 24]",1,4


# Test data

In [13]:
test_data=pd.read_pickle("C:\\Users\\A373503\\Desktop\\Complete_4_models\\HDFS\\Data\\HDFS_test_data.pkl")
test_data.head()

,Blockid,Preprocess_to_log_lines,labels,Length
130892,blk_-7677011170870236960,[_info_dfs_fsnamesystem_block_namesystem_alloc...,0,23
174562,blk_6638067388838485468,[_info_dfs_datanode_dataxceiver_receiving_bloc...,0,25
45990,blk_-2587859430834379342,[_info_dfs_fsnamesystem_block_namesystem_alloc...,0,20
192504,blk_7519767082407935685,[_info_dfs_datanode_dataxceiver_receiving_bloc...,0,19
43736,blk_-2548681213192465965,[_info_dfs_fsnamesystem_block_namesystem_alloc...,0,19


In [14]:
print("the size of the data:",len(test_data))
test_data.groupby(['labels']).count()

the size of the data: 115013


,Blockid,Preprocess_to_log_lines,Length
labels,,,
0,111682,111682,111682
1,3331,3331,3331


In [15]:
print("The size of training data:",len(train_data))
print("The size of test data:",len(test_data))

The size of training data: 460048
The size of test data: 115013


# Here only for the training data

<b> To avoid ignoring the short lengths logs: padding on the data

In [9]:
def padding_short_logs(data,window_size):
    for i in range(len(data)):
        if len(data.EventSequence.iloc[i]) < window_size+1:
            data.EventSequence.iloc[i]=pad_sequences([data.EventSequence.iloc[i]], maxlen=window_size+1)[0]
            #print(data.EventSequence.iloc[i])

In [10]:
data=train_data
window_size=10
padding_short_logs(data,window_size)

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [11]:
train_data[train_data["Length"] < window_size]

,Blockid,EventSequence,labels,Length
549142,blk_9030539814686132531,"[0, 0, 0, 0, 0, 0, 0, 2, 2, 12, 24]",1,4
292808,blk_7717720559218514250,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 32]",1,2
87956,blk_-4397275276774729449,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 27]",1,2
62615,blk_6429121745702481909,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 27]",1,2
52885,blk_5567058207076986877,"[0, 0, 0, 0, 0, 0, 0, 15, 2, 2, 24]",1,4
...,...,...,...,...
7209,blk_7165939739964393997,"[0, 0, 0, 0, 0, 0, 0, 11, 2, 2, 24]",1,4
424987,blk_-6475794039385849761,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 33]",1,2
545982,blk_-6810082241294284349,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 33]",1,2
428480,blk_-2944709241310761499,"[0, 0, 0, 0, 0, 0, 0, 2, 2, 12, 24]",1,4


# Creating the input and target

In [19]:
volab=len(word2id)
def generate_within_blockid(data, window_size,step):
    x = []
    y = []
    seq=[]
    for i in range(len(data)):
        for item in range(0,len(data["EventSequence"].iloc[i])-window_size, step):
            seq=data.EventSequence.iloc[i]
            sentence=seq[item:item+window_size]
            target=seq[item+window_size]
                
            x.append(sentence)
            y.append(target)
            
    x=np.array(x)
    y=to_categorical(y, volab+1)

    return x,y

<b>take an example to look

In [20]:
print("The first sequence is:",list(train_data[:1].EventSequence))

The first sequence is: [[2, 2, 2, 12, 3, 4, 3, 4, 3, 4, 5, 5, 5, 6, 6, 6, 7, 7, 8]]


In [21]:
#then we do the function only on the first sequence
X_n,y_n=generate_within_blockid(train_data[:1], window_size,1)

In [22]:
X_n

array([[ 2,  2,  2, 12,  3,  4,  3,  4,  3,  4],
       [ 2,  2, 12,  3,  4,  3,  4,  3,  4,  5],
       [ 2, 12,  3,  4,  3,  4,  3,  4,  5,  5],
       [12,  3,  4,  3,  4,  3,  4,  5,  5,  5],
       [ 3,  4,  3,  4,  3,  4,  5,  5,  5,  6],
       [ 4,  3,  4,  3,  4,  5,  5,  5,  6,  6],
       [ 3,  4,  3,  4,  5,  5,  5,  6,  6,  6],
       [ 4,  3,  4,  5,  5,  5,  6,  6,  6,  7],
       [ 3,  4,  5,  5,  5,  6,  6,  6,  7,  7]])

In [23]:
np.argmax(y_n,axis=1)

array([5, 5, 5, 6, 6, 6, 7, 7, 8], dtype=int64)

In [24]:
X,y=generate_within_blockid(train_data, window_size,1)

KeyboardInterrupt: 

In [ ]:
len(X) #this is the size of the input of model

# Fix the testdata: 
<b>transform the logs into integers as the same in the vocab and if unknown logs are found, transfer those into <unknown>.

In [25]:
test_data.head()

,Blockid,Preprocess_to_log_lines,labels,Length
130892,blk_-7677011170870236960,[_info_dfs_fsnamesystem_block_namesystem_alloc...,0,23
174562,blk_6638067388838485468,[_info_dfs_datanode_dataxceiver_receiving_bloc...,0,25
45990,blk_-2587859430834379342,[_info_dfs_fsnamesystem_block_namesystem_alloc...,0,20
192504,blk_7519767082407935685,[_info_dfs_datanode_dataxceiver_receiving_bloc...,0,19
43736,blk_-2548681213192465965,[_info_dfs_fsnamesystem_block_namesystem_alloc...,0,19


In [26]:
word2id

{True: 1,
 '_info_dfs_datanode_dataxceiver_receiving_block_blk_src_dest_,': 2,
 '_info_dfs_datanode_packetresponder_packetresponder_for_block_blk_terminating,': 3,
 '_info_dfs_datanode_packetresponder_received_block_blk_of_size_from_,': 4,
 '_info_dfs_fsnamesystem_block_namesystem_addstoredblock_blockmap_updated_is_added_to_blk_size_,': 5,
 '_info_dfs_fsnamesystem_block_namesystem_delete_blk_is_added_to_invalidset_of_,': 6,
 '_info_dfs_fsdataset_deleting_block_blk_file_mnt_hadoop_dfs_data_current_subdir_blk_,': 7,
 '_info_dfs_fsdataset_deleting_block_blk_file_mnt_hadoop_dfs_data_current_subdir_blk_': 8,
 '_info_dfs_datanode_dataxceiver_served_block_blk_to_,': 9,
 '_warn_dfs_datanode_dataxceiver_got_exception_while_serving_blk_to_,': 10,
 '_info_dfs_fsnamesystem_block_namesystem_allocateblock_user_root_rand_temporary_task_m_part_blk_,': 11,
 '_info_dfs_fsnamesystem_block_namesystem_allocateblock_user_root_randtxt_temporary_task_m_part_blk_,': 12,
 '_info_dfs_datablockscanner_verificatio

In [28]:
inx=()
forj=[]
ends=[]
for i in range(len(test_data)):

    lists=test_data.Preprocess_to_log_lines.iloc[i]

    for j in range(len(lists)):
        
        char=test_data.Preprocess_to_log_lines.iloc[i][j]
        if char in word2id:
            inx = word2id[char]
        else:
            inx = 98
            
        forj += [inx]
        
    ends+=[forj]
    forj= []

In [29]:
test_data['EventSequence']=ends

In [30]:
test_data.head()

,Blockid,Preprocess_to_log_lines,labels,Length,EventSequence
130892,blk_-7677011170870236960,[_info_dfs_fsnamesystem_block_namesystem_alloc...,0,23,"[11, 2, 2, 2, 5, 5, 5, 3, 4, 3, 4, 3, 4, 10, 1..."
174562,blk_6638067388838485468,[_info_dfs_datanode_dataxceiver_receiving_bloc...,0,25,"[2, 12, 2, 2, 3, 4, 3, 4, 3, 4, 5, 5, 5, 10, 9..."
45990,blk_-2587859430834379342,[_info_dfs_fsnamesystem_block_namesystem_alloc...,0,20,"[15, 2, 2, 2, 5, 5, 5, 3, 4, 3, 4, 3, 4, 13, 6..."
192504,blk_7519767082407935685,[_info_dfs_datanode_dataxceiver_receiving_bloc...,0,19,"[2, 2, 2, 15, 3, 4, 3, 4, 3, 4, 5, 5, 5, 6, 6,..."
43736,blk_-2548681213192465965,[_info_dfs_fsnamesystem_block_namesystem_alloc...,0,19,"[15, 2, 2, 2, 5, 5, 5, 3, 4, 3, 4, 3, 4, 6, 6,..."


In [31]:
test_data.Preprocess_to_log_lines.iloc[0]

['_info_dfs_fsnamesystem_block_namesystem_allocateblock_user_root_rand_temporary_task_m_part_blk_,',
 '_info_dfs_datanode_dataxceiver_receiving_block_blk_src_dest_,',
 '_info_dfs_datanode_dataxceiver_receiving_block_blk_src_dest_,',
 '_info_dfs_datanode_dataxceiver_receiving_block_blk_src_dest_,',
 '_info_dfs_fsnamesystem_block_namesystem_addstoredblock_blockmap_updated_is_added_to_blk_size_,',
 '_info_dfs_fsnamesystem_block_namesystem_addstoredblock_blockmap_updated_is_added_to_blk_size_,',
 '_info_dfs_fsnamesystem_block_namesystem_addstoredblock_blockmap_updated_is_added_to_blk_size_,',
 '_info_dfs_datanode_packetresponder_packetresponder_for_block_blk_terminating,',
 '_info_dfs_datanode_packetresponder_received_block_blk_of_size_from_,',
 '_info_dfs_datanode_packetresponder_packetresponder_for_block_blk_terminating,',
 '_info_dfs_datanode_packetresponder_received_block_blk_of_size_from_,',
 '_info_dfs_datanode_packetresponder_packetresponder_for_block_blk_terminating,',
 '_info_dfs_

In [32]:
test_data.EventSequence.iloc[0]

[11, 2, 2, 2, 5, 5, 5, 3, 4, 3, 4, 3, 4, 10, 10, 9, 13, 6, 6, 6, 7, 7, 8]

<b>check the short lengths again

In [ ]:
test_data[test_data['Length']<10]

In [ ]:
padding_short_logs(test_data,window_size)

In [ ]:
test_data[test_data["Length"] < window_size]

In [ ]:
test_data.to_pickle("transformer_mix_test.pkl")

In [ ]:
X_test,y_test=generate_within_blockid(test_data[:23003], window_size,1)

In [ ]:
len(X_test) #also, the output here is the size of the input data to do testing

# Model 

In [ ]:
model = Sequential()
model.add(Embedding(volab+1, 128, input_length = window_size)) #input shape: [samples, steps, features]
model.add(LSTM(64))
model.add(Dense(128, activation='relu'))
model.add(Dense(volab+1, activation='softmax')) 
print(model.summary())

<b>validation_split</b>: Float between 0 and 1. Fraction of the training data to be used as validation data. The model will set apart this fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch

In [ ]:
optim=tf.keras.optimizers.Adam(learning_rate=0.001)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau( monitor="val_loss", factor=0.5,
           patience=3, verbose=1)
logs = Callback()
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy',tf.keras.metrics.Recall(),tf.keras.metrics.Precision(),tf.keras.metrics.TruePositives(),tf.keras.metrics.TrueNegatives(), tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives()])
model.fit(X, y, batch_size=64, epochs=10,shuffle = True,validation_split = 0.1, callbacks=[reduce_lr])

In [ ]:
# Save model
#model.save('hdfs_05_09_mix_training.txt') #


# load model
model = load_model('hdfs_05_09_mix_training.txt')


# Do not run from here, try the fixed test data as transformer. 2021_05_18

In [ ]:
test_data= pd.read_pickle("C:\\Users\\A373503\\Desktop\\Complete_4_models\\HDFS\\Transformer\\HDFS_test_data_for_all.pkl")
test_data.labels.value_counts() 

In [ ]:
test_data

In [ ]:
word2id['unknown']

In [ ]:
X_test,y_test=generate_within_blockid(test_data, 10,1)

# softmax output

In [ ]:
#softmax output
prediction = model.predict(X_test, verbose=1)

In [ ]:
test_data=test_data[:23003]

# Point the target to the corresponding block

In [ ]:
test_data.head() #overview the dataset again

In [ ]:
test_data[test_data['Length'<11]]

In [ ]:
test_data.labels.value_counts() #here we will see how many anomalies in the testdata

In [ ]:
def labeled_targets(data,window_size):
    window_num=[]
    for i in range(len(data)):        
        window_n=len(data.EventSequence.iloc[i])-window_size
        window_num.append(window_n)         
    return window_num

In [ ]:
window_num=labeled_targets(test_data,window_size)

In [ ]:
#check the functions are correct
print("sum of the window_num",sum(window_num))
print("window num of X_test",len(X_test))
##################################33
#if the two values are same: no error step now

In [ ]:
for i in window_num:
    if i<1:
        print(i)
#make sure we do not have any negetive numbers
#if we have the negetive number: we did not padding the short length logs

#  Deeplog Method : 

In [ ]:
#accuracy overview 
accs=list()

for i in range(1,20):
    preds=(-prediction).argsort()[:,:i]
    truth=(-y_test).argsort()[:,0]
    truth=truth[:, None]
    
    acc=sum([truth[i] in preds[i] for i in range(len(truth))])/len(truth)  
    accs.append(acc)   
    
x=np.arange(1,20)
plt.xlabel('Threshold')
plt.ylabel('Accuracy')
plt.plot(x,accs)

In [ ]:
int(sum([truth[j] in preds[j] for j in range(0,3)])/3)

In [ ]:
import time
start_time = time.time()

TN=0 #ture negative
FP=0 #false positive
FN=0 #false negative
TP=0 #ture positive
precision=0
recall=0
accuracy=0
f1_score=0
left=0
right=0

n=6

preds=(-prediction).argsort()[:,:n]
truth=(-y_test).argsort()[:,0]
truth=truth[:, None]
window_num.append(-1) #solve the first number issue

for i in range(len(test_data)): #for each blocks

    num_preds=window_num[i] #we have how many targets(prediction)

    #print(num_preds, "predicts in the",i+1,"th block " )
    #print("In this block, the lable should be",test_data_small.Labels.iloc[i])   
    window_num[-1]=-1
    left+=window_num[i-1]
    right+=window_num[i]
       
    if int(sum([truth[j] in preds[j] for j in range(left+1,right)])/window_num[i]) is not 1: # anomaly prediction
        if int(test_data.labels.iloc[i]) == 1: #label is anomaly
            
            TP+=1
        else: #label is normal
            FP+=1
            
    if int(sum([truth[j] in preds[j] for j in range(left+1,right)])/window_num[i]) is 1: #normal prediction
        if int(test_data.labels.iloc[i]) == 0: #label is normal
            TN+=1
        else: #label is anomaly
            FN+=1
            
            

    #here is to check all things are correct!!
    #print(window_num[i],left+1,right)
    if i == len(test_data)-1: 
        print("This print is to check all ranges are correct:")
        print("The range will be in",left+1,'and',right,"and the lenth of the targets in this block is",window_num[i])
        print(" Round",i+1," done!!!\n")

    

                 
        

print("Top",n,":")
#print("We have",len(test_data),"blocks in total.")
#print("We have",len(X_test),"windows in total.")
print("True positive (anomaly label with anomaly prediction):",TP)
print("False positive (normal label with anomaly prediction):",FP)
print("False negative (anomaly label with normal prediction): ",FN)
print("True negative (normal label with normal prediction):",TN)
            
accuracy=(TP+TN)/(TP+TN+FP+FN)
precision=TP/(TP+FP)
recall=TP/(TP+FN)
f1_score=(2*recall*precision)/(recall+precision)

print("Accuracy:",accuracy)
print("Precision:",precision)
print("Recall:",recall)
print("F1 score:",f1_score)
print("---The runing time is: %s seconds ---" % (time.time() - start_time))         


This print is to check all ranges are correct:
The range will be in 1097453 and 1097462 and the lenth of the targets in this block is 9
 Round 115013  done!!!

Top 6 :
True positive (anomaly label with anomaly prediction): 149
False positive (normal label with anomaly prediction): 130
False negative (anomaly label with normal prediction):  3182
True negative (normal label with normal prediction): 111552
Accuracy: 0.9712032552841853
Precision: 0.5340501792114696
Recall: 0.04473131191834284
F1 score: 0.08254847645429364
---The runing time is: 13.718751668930054 seconds ---

This print is to check all ranges are correct:
The range will be in 218508 and 218521 and the lenth of the targets in this block is 13
 Round 23003  done!!!

Top 6 :
True positive (anomaly label with anomaly prediction): 654
False positive (normal label with anomaly prediction): 436
False negative (anomaly label with normal prediction):  2
True negative (normal label with normal prediction): 21911
Accuracy: 0.9809590053471287
Precision: 0.6
Recall: 0.9969512195121951
F1 score: 0.7491408934707904
---The runing time is: 2.5261642932891846 seconds ---